In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import accuracy_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
%run Pre_Processing.ipynb

In [3]:
train = pd.read_csv("train.csv", index_col='id')
test = pd.read_csv("test.csv", index_col='id')
submission = pd.read_csv("sample_submission.csv")

df = train.copy()

target = "NObeyesdad"

In [4]:
X = df.drop([target], axis=1)
y = df[target]

In [5]:
target_mapping = {
    "Insufficient_Weight" : 0,
    "Normal_Weight" : 1,
    "Overweight_Level_I" : 2,
    "Overweight_Level_II" : 3,
    "Obesity_Type_I" : 4,
    "Obesity_Type_II" : 5,
    "Obesity_Type_III" : 6
}

y_mapped = y.map(target_mapping)
y_reverse_map = {v : k for k, v in target_mapping.items()}

In [6]:
preprocessed_train = preprocessor.fit_transform(X)
preprocessed_train_df = pd.DataFrame(preprocessed_train, columns=preprocessor.get_feature_names_out())

In [7]:
preprocessed_test = preprocessor.transform(test)
preprocessed_test_df = pd.DataFrame(preprocessed_test, columns=preprocessor.get_feature_names_out())

In [8]:
def accuracy_eval(y_true, y_pred):
    """
    Custom evaluation function to compute accuracy.
    """
    y_pred_labels = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(y_true, y_pred_labels)
    return 'accuracy', accuracy, True

## Optuna

### LGBM

In [27]:
import optuna
from lightgbm import early_stopping
from optuna.integration import LightGBMPruningCallback

def objective(trial, X_train, y_train):
    
    params = {
        'learning_rate' : trial.suggest_float('learning_rate', .001, .1, log = True),
        'max_depth' : trial.suggest_int('max_depth', 2, 20),
        'subsample' : trial.suggest_float('subsample', .5, 1),
        'min_child_weight' : trial.suggest_float('min_child_weight', .1, 15, log = True),
        'reg_lambda' : trial.suggest_float('reg_lambda', .1, 20, log = True),
        'reg_alpha' : trial.suggest_float('reg_alpha', .1, 10, log = True),
        'n_estimators' : trial.suggest_int('n_estimators', 5, 1000),
        'random_state' : seed,
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
    }
    
    X_train.reset_index(drop=True, inplace=True)    
    y_train.reset_index(drop=True, inplace=True)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[test_idx]

        model = LGBMClassifier(**param_grid, verbose=-1)
        model.fit(
            X_train_fold,
            y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            eval_metric=accuracy_eval,   
            callbacks=[early_stopping(100)]
        )
        preds = model.predict_proba(X_test)
        y_pred_labels = np.argmax(preds, axis=1)
        cv_scores[idx] = accuracy_score(y_test, y_pred_labels)
    
    return np.mean(cv_scores)

In [28]:
X_train, X_test, y_train, y_test = train_test_split(preprocessed_train_df, y_mapped, train_size=0.8, test_size=0.2, random_state=0)

In [29]:
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X_train, y_train)
study.optimize(func, timeout=3*3600)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2024-02-13 15:22:10,479] A new study created in memory with name: LGBM Classifier


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[14]	valid_0's multi_logloss: 0.479201	valid_0's accuracy: 0.836845
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[14]	valid_0's multi_logloss: 0.458112	valid_0's accuracy: 0.823848
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[14]	valid_0's multi_logloss: 0.459329	valid_0's accuracy: 0.835893
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[14]	valid_0's multi_logloss: 0.458015	valid_0's accuracy: 0.840711
Training until validation scores don't improve for 100 rounds


[I 2024-02-13 15:22:11,449] Trial 0 finished with value: 0.8393063583815028 and parameters: {'n_estimators': 15, 'learning_rate': 0.8543008736866858, 'num_leaves': 1006, 'max_depth': 16, 'min_data_in_leaf': 49, 'colsample_bytree': 0.8040396077088546, 'subsample': 1.1041155258547837, 'min_gain_to_split': 14.679311205118804, 'bagging_fraction': 0.6000000000000001, 'bagging_freq': 1, 'feature_fraction': 0.2}. Best is trial 0 with value: 0.8393063583815028.


Did not meet early stopping. Best iteration is:
[14]	valid_0's multi_logloss: 0.459457	valid_0's accuracy: 0.838302
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[268]	valid_0's multi_logloss: 0.681966	valid_0's accuracy: 0.860927
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[268]	valid_0's multi_logloss: 0.685305	valid_0's accuracy: 0.859982
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[268]	valid_0's multi_logloss: 0.684673	valid_0's accuracy: 0.859079
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[268]	valid_0's multi_logloss: 0.688928	valid_0's accuracy: 0.854863
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[268]	valid_0's multi_logloss: 0.684633	valid_0's accuracy

[I 2024-02-13 15:22:26,988] Trial 1 finished with value: 0.8641618497109826 and parameters: {'n_estimators': 268, 'learning_rate': 0.007061538521341047, 'num_leaves': 729, 'max_depth': 9, 'min_data_in_leaf': 87, 'colsample_bytree': 0.8344146937976558, 'subsample': 1.010575206324131, 'min_gain_to_split': 12.473135941539713, 'bagging_fraction': 0.5, 'bagging_freq': 1, 'feature_fraction': 0.30000000000000004}. Best is trial 1 with value: 0.8641618497109826.


...

Early stopping, best iteration is:
[33]	valid_0's multi_logloss: 0.557946	valid_0's accuracy: 0.897019
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[181]	valid_0's multi_logloss: 0.289896	valid_0's accuracy: 0.908489
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[132]	valid_0's multi_logloss: 0.269881	valid_0's accuracy: 0.916591
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[201]	valid_0's multi_logloss: 0.264611	valid_0's accuracy: 0.910569
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[191]	valid_0's multi_logloss: 0.278286	valid_0's accuracy: 0.906052
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[148]	valid_0's multi_logloss: 0.277915	valid_0's accuracy: 0.912978


[I 2024-02-13 18:22:08,263] Trial 268 finished with value: 0.9065992292870906 and parameters: {'n_estimators': 781, 'learning_rate': 0.02825017343199187, 'num_leaves': 729, 'max_depth': 19, 'min_data_in_leaf': 24, 'colsample_bytree': 0.41345059640113496, 'subsample': 1.2084810637396006, 'min_gain_to_split': 0.2583782057309001, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.5}. Best is trial 75 with value: 0.9079961464354529.


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[286]	valid_0's multi_logloss: 0.291673	valid_0's accuracy: 0.910295
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[379]	valid_0's multi_logloss: 0.249032	valid_0's accuracy: 0.91629
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[285]	valid_0's multi_logloss: 0.268007	valid_0's accuracy: 0.911774
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[300]	valid_0's multi_logloss: 0.276917	valid_0's accuracy: 0.90816
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[280]	valid_0's multi_logloss: 0.27296	valid_0's accuracy: 0.91358


[I 2024-02-13 18:23:17,189] Trial 269 finished with value: 0.9066473988439308 and parameters: {'n_estimators': 889, 'learning_rate': 0.016889133933421223, 'num_leaves': 348, 'max_depth': 18, 'min_data_in_leaf': 55, 'colsample_bytree': 0.5521681542949233, 'subsample': 1.174671001952609, 'min_gain_to_split': 0.012161019583127597, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.5}. Best is trial 75 with value: 0.9079961464354529.


Number of finished trials: 270
Best trial: {'n_estimators': 745, 'learning_rate': 0.015814527100162998, 'num_leaves': 742, 'max_depth': 19, 'min_data_in_leaf': 18, 'colsample_bytree': 0.8309728636518279, 'subsample': 1.1554573550007061, 'min_gain_to_split': 1.1021958438092825, 'bagging_fraction': 0.8, 'bagging_freq': 1, 'feature_fraction': 0.5}


In [30]:
print(f"\tBest value (Accuracy): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (Accuracy): 0.90800
	Best params:
		n_estimators: 745
		learning_rate: 0.015814527100162998
		num_leaves: 742
		max_depth: 19
		min_data_in_leaf: 18
		colsample_bytree: 0.8309728636518279
		subsample: 1.1554573550007061
		min_gain_to_split: 1.1021958438092825
		bagging_fraction: 0.8
		bagging_freq: 1
		feature_fraction: 0.5


In [31]:
lgbm_params = {k: v for k, v in study.best_params.items()}
lgbm_params

{'n_estimators': 745,
 'learning_rate': 0.015814527100162998,
 'num_leaves': 742,
 'max_depth': 19,
 'min_data_in_leaf': 18,
 'colsample_bytree': 0.8309728636518279,
 'subsample': 1.1554573550007061,
 'min_gain_to_split': 1.1021958438092825,
 'bagging_fraction': 0.8,
 'bagging_freq': 1,
 'feature_fraction': 0.5}

### XGBoost

In [70]:
def objective(trial, X_train, y_train):
    
    param_grid = {
        'random_state': 42,
        "n_estimators": trial.suggest_int("n_estimators", 10, 1000),
        #'reg_alpha': trial.suggest_float('reg_alpha', 1e-3, 10.0, log=True),
        #'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0, log=True),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.05, 1),
        "subsample": trial.suggest_float("subsample", 0.05, 1.0),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1, log=True),
        "max_depth": trial.suggest_int("max_depth", 3, 20),
    }
    
    X_train.reset_index(drop=True, inplace=True)    
    y_train.reset_index(drop=True, inplace=True)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[test_idx]

        model = XGBClassifier(**param_grid)
        model.fit(
            X_train_fold,
            y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            eval_metric=accuracy_eval,   
            early_stopping_rounds=100       
        )
        preds = model.predict_proba(X_test)
        y_pred_labels = np.argmax(preds, axis=1)
        cv_scores[idx] = accuracy_score(y_test, y_pred_labels)
    
    return np.mean(cv_scores)

In [ ]:
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X_train, y_train)
study.optimize(func, timeout=3600)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
print(f"\tBest value (Accuracy): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

### Catboost

In [ ]:
import optuna
from lightgbm import early_stopping
from optuna.integration import LightGBMPruningCallback

def objective(trial, X_train, y_train):
    
    param_grid = {
        'iterations' : trial.suggest_int("iterations", 100, 1000),
        'learning_rate' : trial.suggest_float("learning_rate", 1e-3, 1.5, log=True),
        'depth' : trial.suggest_int("depth", 4, 20),
        'l2_leaf_reg' : trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        #'bootstrap_type' : trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        'random_strength' : trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        'bagging_temperature' : trial.suggest_float("bagging_temperature", 0.0, 10.0),
        'od_type' : trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        'od_wait' : trial.suggest_int("od_wait", 10, 50),
        'verbose' : False
    }
    
    X_train.reset_index(drop=True, inplace=True)    
    y_train.reset_index(drop=True, inplace=True)
    
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    cv_scores = np.empty(5)
    
    for idx, (train_idx, test_idx) in enumerate(cv.split(X_train, y_train)):
        X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[test_idx]
        y_train_fold, y_val_fold = y_train[train_idx], y_train[test_idx]

        model = CatBoostClassifier(**param_grid)
        model.fit(
            X_train_fold,
            y_train_fold,
            eval_set=[(X_val_fold, y_val_fold)],
            eval_metric=accuracy_eval,   
            callbacks=[early_stopping(100)]
        )
        preds = model.predict_proba(X_test)
        y_pred_labels = np.argmax(preds, axis=1)
        cv_scores[idx] = accuracy_score(y_test, y_pred_labels)
    
    return np.mean(cv_scores)

In [ ]:
study = optuna.create_study(direction="maximize", study_name="LGBM Classifier")
func = lambda trial: objective(trial, X_train, y_train)
study.optimize(func, timeout=3600)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
print(f"\tBest value (Accuracy): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

## Model Training and submission

In [12]:
from sklearn.ensemble import VotingClassifier

In [18]:
dart_params = {
    'random_state' : 42,
    'n_estimators': 5000, 
    'reg_alpha': 2.156916332573565, 
    'reg_lambda': 0.0020442251900243155, 
    'colsample_bytree': 0.4, 
    'subsample': 0.4, 
    'learning_rate': 0.008, 
    'max_depth': 100, 
    'num_leaves': 92, 
    'min_child_samples': 86, 
    'min_data_per_groups': 61,
    'boosting_type' : 'dart',
    'verbose': -1
}

xgb_params = {
    'random_state' : 42,
    'n_estimators': 20000, 
    'reg_alpha': 1.8139033219985152, 
    'reg_lambda': 1.68320446559976, 
    'colsample_bytree': 0.3, 
    'subsample': 0.5, 
    'learning_rate': 0.008, 
    'max_depth': 10
}

cat_params = {
    'random_state': 42,
    'iterations': 467, 
    'learning_rate': 0.07306038997735301, 
    'depth': 5, 
    'l2_leaf_reg': 0.20454589133303308, 
    'bootstrap_type': 'Bayesian', 
    'random_strength': 5.592253851702421e-06,
    'bagging_temperature': 0.362583079852707, 
    'od_type': 'Iter', 
    'od_wait': 40
}

# lgbm_params = {
#     'n_estimators': 443,
#     'learning_rate': 0.017706677698972367,
#     'num_leaves': 765,
#     'max_depth': 11,
#     'min_data_in_leaf': 63,
#     'max_bin': 266,
#     'colsample_bytree': 0.44058514304002716,
#     'subsample': 0.847981015269405,
#     'min_gain_to_split': 0.44554943848909184
# }

In [14]:
dart = LGBMClassifier(**dart_params)
xgb = XGBClassifier(**xgb_params)
cat = CatBoostClassifier(**cat_params)

ensemble = VotingClassifier(
    [
        ("LightGBM", dart),
        ("XGBoost", xgb),
        ("CatBoost", cat)
    ],
    voting="soft",
    weights=(0.6, 0.2, 0.2),
)


In [32]:
model = LGBMClassifier(**lgbm_params, verbose=-1)

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.9079961464354528


In [33]:
model.fit(preprocessed_train_df, y_mapped)

LGBMClassifier(bagging_fraction=0.8, bagging_freq=1,
               colsample_bytree=0.8309728636518279, feature_fraction=0.5,
               learning_rate=0.015814527100162998, max_depth=19,
               min_data_in_leaf=18, min_gain_to_split=1.1021958438092825,
               n_estimators=745, num_leaves=742, subsample=1.1554573550007061,
               verbose=-1)

In [34]:
preds = model.predict(preprocessed_test_df)

In [35]:
y_pred_inv = np.vectorize(y_reverse_map.get)(preds)
y_pred_inv

array(['Obesity_Type_II', 'Overweight_Level_I', 'Obesity_Type_III', ...,
       'Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_II'],
      dtype='<U19')

In [36]:
submission["NObeyesdad"] = y_pred_inv
submission.to_csv("./preds/lgbm_optuna_3h.csv", index=False)

## Cross Validation

In [37]:
seed = 100
splits = 5
repeats = 4

rskf = RepeatedStratifiedKFold(n_splits=splits, random_state=seed, n_repeats=repeats)
skf = StratifiedKFold(n_splits=splits, random_state=seed, shuffle=True)

np.random.seed(seed)

X = preprocessed_train_df.to_numpy()
y = y_mapped.to_numpy()

models = []
accuracies = []

for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    model = LGBMClassifier(random_state=seed, verbose=-1)

    model.fit(X_train, y_train)

    models.append(model)
    
    y_pred = model.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred) 
    accuracies.append(accuracy)
    
    print(f"Fold {fold} Accuracy: {accuracy}")
    
mean_accuracy = np.mean(accuracies)
print(f"Mean Accuracy: {mean_accuracy}")

Fold 1 Accuracy: 0.9024566473988439
Fold 2 Accuracy: 0.911849710982659
Fold 3 Accuracy: 0.9067919075144508
Fold 4 Accuracy: 0.9021922428330523
Fold 5 Accuracy: 0.906046735726331
Mean Accuracy: 0.9058674488910674


In [39]:
models[0].predict_proba(preprocessed_test_df).shape

(13840, 7)

In [87]:
probabilities = []

for model in models:
    proba = model.predict_proba(preprocessed_test_df)
    probabilities.append(proba)
    
mean_probabilities = np.mean(probabilities, axis=0)

final_predictions = np.argmax(mean_probabilities, axis=1)

In [88]:
final_predictions

array([5, 2, 6, ..., 0, 1, 5], dtype=int64)

In [89]:
y_pred_inv = np.vectorize(y_reverse_map.get)(final_predictions)
y_pred_inv

array(['Obesity_Type_II', 'Overweight_Level_I', 'Obesity_Type_III', ...,
       'Insufficient_Weight', 'Normal_Weight', 'Obesity_Type_II'],
      dtype='<U19')

In [90]:
submission["NObeyesdad"] = y_pred_inv
submission.to_csv("./preds/pred5.csv", index=False)

- train_test_split with random_state=0 gave a better result between the validation and submission score than the cross validation